<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task1/blob/main/Data_augmetation_500_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlpaug
!pip install transformers


In [ ]:
import nlpaug.augmenter.word as naw

# Ejemplo de DataFrame con columnas "text" y "language" que contiene sentencias en inglés y español
import pandas as pd
data = pd.DataFrame({
    'text': ['This is an English sentence.', 'Esto es una oración en español.'],
    'language': ['english', 'spanish']
})

# Crear instancias de aumentador de texto para inglés y español
aug_english = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
aug_spanish = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")

# Aplicar aumentación de texto para cada idioma
augmented_texts = []
for idx, row in data.iterrows():
    if row['language'] == 'english':
        augmented_text = aug_english.augment(row['text'])
    elif row['language'] == 'spanish':
        augmented_text = aug_spanish.augment(row['text'])
    else:
        augmented_text = row['text']  # Mantener el texto sin cambios si el idioma no es inglés ni español
    augmented_texts.append(augmented_text)

# Agregar los textos aumentados al DataFrame
data['augmented_text'] = augmented_texts


In [ ]:
data.head()


,text,language,augmented_text
0,This is an English sentence.,english,[this is an early english conditional sentence.]
1,Esto es una oración en español.,spanish,[Esto es es una oración que en latín español.]


In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t')


In [ ]:
!pip install nlpaug
!pip install transformers

In [ ]:
import nlpaug.augmenter.word as naw
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd

# Cargar el DataFrame con las sentencias y las clases objetivo
# Supongamos que las columnas se llaman 'text' y 'language', y 'task1' contiene las clases objetivo
# Ejemplo de DataFrame con 4000 filas
# data = pd.read_csv('data.csv')

# Cargar el modelo BERT pre-entrenado multilingüe
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()

# Crear instancias de aumentador de texto para inglés y español
aug_english = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")
aug_spanish = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")

# Función para aplicar aumentación de texto a una oración
def augment_text(text, language):
    if language == 'en':
        augmented_text = aug_english.augment(text)
    elif language == 'es':
        augmented_text = aug_spanish.augment(text)
    else:
        augmented_text = text  # Mantener el texto sin cambios si el idioma no es inglés ni español
    return augmented_text

# Crear listas para almacenar las sentencias y clases objetivo aumentadas
augmented_texts = []
augmented_languages = []
classes = []

# Recorrer el DataFrame y aplicar aumentación a cada fila
for idx, row in df_train.iterrows():
    text = row['text']
    language = row['language']
    target_class = row['task1']

    # Aplicar aumentación de texto en el idioma correspondiente
    augmented_text_1 = augment_text(text, language)
    augmented_text_2 = augment_text(text, language)

    # Agregar las sentencias y clases objetivo aumentadas a las listas
    augmented_texts.extend([augmented_text_1, augmented_text_2])
    augmented_languages.extend([language, language])
    classes.extend([target_class, target_class])

# Crear un nuevo DataFrame con las sentencias y clases objetivo aumentadas
augmented_data = pd.DataFrame({
    'text': augmented_texts,
    'language': augmented_languages,
    'task1': classes
})

# Verificar el nuevo DataFrame con las sentencias y clases objetivo aumentadas
print(augmented_data.head())


In [ ]:
import nlpaug.augmenter.word as naw
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd

# Cargar el DataFrame con las sentencias y las clases objetivo
# Supongamos que las columnas se llaman 'text' y 'language', y 'task1' contiene las clases objetivo
# Ejemplo de DataFrame con 4000 filas
# data = pd.read_csv('data.csv')

# Cargar el modelo BERT pre-entrenado multilingüe
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()

# Crear instancias de aumentador de texto para inglés y español
aug_english = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")
aug_spanish = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")

# Función para aplicar aumentación de texto a una oración
def augment_text(text, language):
    if language == 'en':
        augmented_text = aug_english.augment(text)
    elif language == 'es':
        augmented_text = aug_spanish.augment(text)
    else:
        augmented_text = text  # Mantener el texto sin cambios si el idioma no es inglés ni español
    return augmented_text

# Crear listas para almacenar las sentencias y clases objetivo aumentadas
augmented_texts = []
augmented_languages = []
classes = []

# Variables para contar el número de sentencias aumentadas por idioma
sentences_per_language = {}
max_sentences_per_language = 500  # Limitar a 500 sentencias por idioma

# Recorrer el DataFrame y aplicar aumentación a cada fila
for idx, row in df_train.iterrows():
    text = row['text']
    language = row['language']
    target_class = row['task1']

    # Verificar si ya se han aumentado suficientes sentencias para este idioma
    if language in sentences_per_language and sentences_per_language[language] >= max_sentences_per_language:
        continue

    # Aplicar aumentación de texto en el idioma correspondiente
    augmented_text = augment_text(text, language)

    # Agregar la sentencia aumentada y la clase objetivo a las listas
    augmented_texts.append(augmented_text)
    augmented_languages.append(language)
    classes.append(target_class)

    # Actualizar el contador de sentencias para el idioma actual
    sentences_per_language[language] = sentences_per_language.get(language, 0) + 1

    # Detener la aumentación para este idioma si se han alcanzado las 500 sentencias
    if sentences_per_language[language] >= max_sentences_per_language:
        continue

# Crear un nuevo DataFrame con las sentencias y clases objetivo aumentadas
augmented_data = pd.DataFrame({
    'text': augmented_texts,
    'language': augmented_languages,
    'task1': classes
})

# Verificar el nuevo DataFrame con las sentencias y clases objetivo aumentadas
print(augmented_data.head())
#1 h 2 min 59 s

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                                text language       task1
0  [2011 She calls herself " English anti - femin...       en      sexist
1  [Now, back there to these women, the brave and...       en  non-sexist
2  [@ Planet CurvyBandida @ Space Xalynne _ B Wow...       en      sexist
3  [@ AurelieGuiboud Incredible! Beautiful! Lonel...       en  non-sexist
4  [i find it extremely hard to believe that kell...       en  non-sexist


In [ ]:
import pandas as pd

# Especificar el nombre del archivo CSV
file_path = '/content/synonym_augmented_data.csv'  # Reemplaza 'ruta/al/archivo.csv' con la ruta real de tu archivo CSV

# Leer el archivo CSV en un DataFrame usando el separador de tabulador
augmented_data = pd.read_csv(file_path, sep='\t')

# Extraer las columnas 'text', 'language' y 'task1' usando expresiones regulares
augmented_data[['text', 'language', 'task1']] = augmented_data['text,language,task1'].str.extract(r"^\['([^']+)',\s*'([^']+)',\s*'([^']+)'\]$")

# Eliminar los espacios adicionales en las columnas 'language' y 'task1'
augmented_data['language'] = augmented_data['language'].str.strip()
augmented_data['task1'] = augmented_data['task1'].str.strip()

# Verificar el DataFrame cargado con las columnas separadas y formateadas correctamente
print(augmented_data.head())




                                 text,language,task1 text language task1
0  ['2011 She calls herself " English anti - femi...  NaN      NaN   NaN
1  ['Now, back there to these women, the brave an...  NaN      NaN   NaN
2  ["@ Planet CurvyBandida @ Space Xalynne _ B Wo...  NaN      NaN   NaN
3  ['@ AurelieGuiboud Incredible! Beautiful! Lone...  NaN      NaN   NaN
4  ['i find it extremely hard to believe that kel...  NaN      NaN   NaN


In [ ]:
import pandas as pd

# Especificar el nombre del archivo CSV
file_path = '/content/synonym.csv'  # Reemplaza 'ruta/al/archivo.csv' con la ruta real de tu archivo CSV



## Leer el archivo CSV en un DataFrame usando el separador de tabulador
augmented_data = pd.read_csv(file_path, sep='\t')

print(augmented_data.head())

                                                text language       task1
0  2011 She calls herself " English anti - femina...       en      sexist
1  ,Now,, back there to these women,, the brave a...       en  non-sexist
2  ,@ Planet CurvyBandida @ Space Xalynne _ B Wow...       en      sexist
3  @ AurelieGuiboud Incredible! Beautiful! Lonely...       en  non-sexist
4  ,i find it extremely hard to believe that kell...       en  non-sexist


In [ ]:
import pandas as pd

# Especificar el nombre del archivo CSV
file_path = '/content/synonym_augmented_data_ok.csv'   # Reemplaza 'ruta/al/archivo.csv' con la ruta real de tu archivo CSV

# Leer el archivo CSV en un DataFrame usando el separador de tabulador
augmented_data = pd.read_csv(file_path, sep='\t')

# Eliminar los corchetes '[' y ']' al inicio y final de cada sentencia en la columna 'text,language,task1'
augmented_data['text,language,task1'] = augmented_data['text,language,task1'].str.strip("[]")

# Dividir la columna 'text,language,task1' en tres columnas separadas 'text', 'language' y 'task1'
augmented_data[['text', 'language', 'task1']] = augmented_data['text,language,task1'].str.split(",", expand=True)

# Eliminar los espacios adicionales en las columnas 'text', 'language' y 'task1'
augmented_data['text'] = augmented_data['text'].str.strip()
augmented_data['language'] = augmented_data['language'].str.strip()
augmented_data['task1'] = augmented_data['task1'].str.strip()

# Eliminar la columna 'text,language,task1', ya que ya hemos dividido los valores en columnas separadas
augmented_data.drop(columns='text,language,task1', inplace=True)

# Verificar el DataFrame cargado con las columnas separadas y formateadas correctamente
print(augmented_data.head())


ValueError: ignored

In [ ]:
# Guardar el DataFrame aumentado en un archivo CSV
augmented_data.to_csv('synonym_augmented_data.csv', index=False)  # index=False para no guardar el índice del DataFrame


In [ ]:
# Obtener la longitud del DataFrame (número de filas)
length_augmented_data = len(augmented_data)
print("Longitud del DataFrame 'augmented_data':", length_augmented_data)

# Ver información general sobre el DataFrame
print(augmented_data.info())


Longitud del DataFrame 'augmented_data': 1000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1000 non-null   object
 1   language  1000 non-null   object
 2   task1     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
None


In [ ]:
augmented_data.head(550)

,text,language,task1
0,"[2011 She calls herself "" English anti - femin...",en,sexist
1,"[Now, back there to these women, the brave and...",en,non-sexist
2,[@ Planet CurvyBandida @ Space Xalynne _ B Wow...,en,sexist
3,[@ AurelieGuiboud Incredible! Beautiful! Lonel...,en,non-sexist
4,[i find it extremely hard to believe that kell...,en,non-sexist
...,...,...,...
545,[busco mujer q qui quiera ser felizrequisistos...,es,sexist
546,[[ Pedicabo ego vos libero et irrumabo ] es as...,es,non-sexist
547,"[Además es no entiendo, porque se legalice tod...",es,non-sexist
548,[Es impresionante como algo é tan tan fantásti...,es,non-sexist
